In [1]:
import csv
import time
import traceback

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import traceback
from copy import deepcopy

from kiteconnect import KiteConnect

from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
warnings.filterwarnings('ignore')
import math
import redis
import six
import sys
import time
import json
import struct
import logging
import threading
from datetime import datetime, timedelta
from access_token import access_token
# from .__version__ import __version__, __title__


Successful Login with Request Token:oyIvFL1rEzh3O3EHRnZt02WnWBQzNPkH


2023-11-29 13:59:59.512 
  command:

    streamlit run c:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


hdatmJeIF3mMd8MwUeRPu3mc9pRtwwnc


In [4]:
access_token

'KZ76dEmmYUjBQGlGgJAnfw0Za0Mpb53p'

In [3]:
kite = KiteConnect(api_key='c1zwly326w2c7wrj')
kite.set_access_token(access_token)

In [2]:

def instrumentLookup(symbol):
    global access_token, key_secret, kite, instrument_df, tickers, quantity
    try:
        return instrument_df[instrument_df.tradingsymbol == symbol].instrument_token.values[0]
    except:
        return -1


def getInstruments():
    global access_token, key_secret, kite, instrument_df, tickers, quantity
    while True:
        try:
            instrument_dump = kite.instruments()
            instrument_df = pd.DataFrame(instrument_dump)
            break
        except Exception as e:
            print(str(e))
            continue

In [4]:
getInstruments()
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,537733638,2100522,EURINR23DEC83.25CE,EURINR,0.0,2023-12-27,83.25,0.0025,1,CE,BCD-OPT,BCD
1,537730310,2100509,EURINR23DEC83.25PE,EURINR,0.0,2023-12-27,83.25,0.0025,1,PE,BCD-OPT,BCD
2,537617158,2100067,EURINR23DEC83.5CE,EURINR,0.0,2023-12-27,83.50,0.0025,1,CE,BCD-OPT,BCD
3,537614342,2100056,EURINR23DEC83.5PE,EURINR,0.0,2023-12-27,83.50,0.0025,1,PE,BCD-OPT,BCD
4,537747718,2100577,EURINR23DEC83.75CE,EURINR,0.0,2023-12-27,83.75,0.0025,1,CE,BCD-OPT,BCD


In [8]:
def get_atm_nifty(ltp):
    nifty_step_size = 50
    mod = math.fmod(ltp, nifty_step_size)
    if mod >= 25:
        atm_value = ltp - mod + nifty_step_size
    else:
        atm_value = ltp - mod
    #print('Nifty ATM Value: ', int(atm_value))
    return int(atm_value)

def get_atm_bank_nifty(ltp):
    bnf_step_size = 100
    mod = math.fmod(ltp, bnf_step_size)
    if mod > 50:
        atm_value = int((ltp//100)*100)+100
    else:
        atm_value = int(ltp//100)*100
    return atm_value
    

def fetch_ltp(kite, name):
    '''
    Function takes kite object, name of exchange and name of instruments
    and returns the ltp
    '''
    instrument_name = str(instrumentLookup(name))
    
    ltp = kite.ltp(instrument_name)[instrument_name]['last_price']
    return ltp

In [9]:
print(get_atm_nifty(fetch_ltp(kite, 'NIFTY 50')))
print(fetch_ltp(kite, 'NIFTY 50'))
# print(kite.ltp("256265")["256265"]['last_price'])

19850
19825.05


In [10]:
nifty_df = instrument_df[(instrument_df['name'] == 'NIFTY') & (instrument_df['instrument_type'] == 'CE')& (instrument_df['strike'] == (get_atm_nifty(fetch_ltp(kite, 'NIFTY 50'))))]
nifty_df = nifty_df.head(3)

In [13]:
nifty_df

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,15023106,58684,NIFTY23NOV19800CE,NIFTY,0.0,2023-11-30,19800.0,0.05,50,CE,NFO-OPT,NFO
1,16767746,65499,NIFTY23DEC19800CE,NIFTY,0.0,2023-12-28,19800.0,0.05,50,CE,NFO-OPT,NFO
2,11042818,43136,NIFTY23D0719800CE,NIFTY,0.0,2023-12-07,19800.0,0.05,50,CE,NFO-OPT,NFO
3,14998274,58587,NIFTY23NOV19050CE,NIFTY,0.0,2023-11-30,19050.0,0.05,50,CE,NFO-OPT,NFO
4,16600578,64846,NIFTY23DEC19050CE,NIFTY,0.0,2023-12-28,19050.0,0.05,50,CE,NFO-OPT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
88,16783106,65559,NIFTY23DEC20450CE,NIFTY,0.0,2023-12-28,20450.0,0.05,50,CE,NFO-OPT,NFO
89,11057410,43193,NIFTY23D0720450CE,NIFTY,0.0,2023-12-07,20450.0,0.05,50,CE,NFO-OPT,NFO
90,15042818,58761,NIFTY23NOV20500CE,NIFTY,0.0,2023-11-30,20500.0,0.05,50,CE,NFO-OPT,NFO
91,16783618,65561,NIFTY23DEC20500CE,NIFTY,0.0,2023-12-28,20500.0,0.05,50,CE,NFO-OPT,NFO


In [12]:
# 15 itm/otm CE
for i in range(-15, 15):
    temp_df = instrument_df[(instrument_df['name'] == 'NIFTY') & (instrument_df['instrument_type'] == 'CE')& (instrument_df['strike'] == (get_atm_nifty(fetch_ltp(kite, 'NIFTY 50'))) + (50*i))]
    temp_df = temp_df.head(3)
    nifty_df = pd.concat([nifty_df, temp_df], ignore_index=True)

In [14]:
nifty_df_pe = instrument_df[(instrument_df['name'] == 'NIFTY') & (instrument_df['instrument_type'] == 'PE')& (instrument_df['strike'] == (get_atm_nifty(fetch_ltp(kite, 'NIFTY 50'))))]
nifty_df_pe = nifty_df_pe.head(3)


In [15]:
# 15 itm/otm PE
for i in range(-15, 15):
    temp_df_pe = instrument_df[(instrument_df['name'] == 'NIFTY') & (instrument_df['instrument_type'] == 'PE')& (instrument_df['strike'] == (get_atm_nifty(fetch_ltp(kite, 'NIFTY 50'))) + (50*i))]
    temp_df_pe = temp_df_pe.head(3)
    nifty_df_pe = pd.concat([nifty_df_pe, temp_df_pe], ignore_index=True)

In [16]:
nifty_df = pd.concat([nifty_df, nifty_df_pe], ignore_index=True)

In [17]:
nifty_df

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,15023106,58684,NIFTY23NOV19800CE,NIFTY,0.0,2023-11-30,19800.0,0.05,50,CE,NFO-OPT,NFO
1,16767746,65499,NIFTY23DEC19800CE,NIFTY,0.0,2023-12-28,19800.0,0.05,50,CE,NFO-OPT,NFO
2,11042818,43136,NIFTY23D0719800CE,NIFTY,0.0,2023-12-07,19800.0,0.05,50,CE,NFO-OPT,NFO
3,14998274,58587,NIFTY23NOV19050CE,NIFTY,0.0,2023-11-30,19050.0,0.05,50,CE,NFO-OPT,NFO
4,16600578,64846,NIFTY23DEC19050CE,NIFTY,0.0,2023-12-28,19050.0,0.05,50,CE,NFO-OPT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
181,16783362,65560,NIFTY23DEC20450PE,NIFTY,0.0,2023-12-28,20450.0,0.05,50,PE,NFO-OPT,NFO
182,11057666,43194,NIFTY23D0720450PE,NIFTY,0.0,2023-12-07,20450.0,0.05,50,PE,NFO-OPT,NFO
183,15043074,58762,NIFTY23NOV20500PE,NIFTY,0.0,2023-11-30,20500.0,0.05,50,PE,NFO-OPT,NFO
184,16783874,65562,NIFTY23DEC20500PE,NIFTY,0.0,2023-12-28,20500.0,0.05,50,PE,NFO-OPT,NFO


In [18]:
token_list = nifty_df['instrument_token'].tolist()
token_list

[15023106,
 16767746,
 11042818,
 14998274,
 16600578,
 11035138,
 14998786,
 16602114,
 11035650,
 14999298,
 16606210,
 11036162,
 14999810,
 16606722,
 11036674,
 15000322,
 16733954,
 11037186,
 15000834,
 16734466,
 11037698,
 15015938,
 16735490,
 11038210,
 15016450,
 16736002,
 11038722,
 15016962,
 16736514,
 11039234,
 15017474,
 16749058,
 11039746,
 15019522,
 16759298,
 11040258,
 15020034,
 16759810,
 11040770,
 15020546,
 16760834,
 11041282,
 15022082,
 16761346,
 11041794,
 15022594,
 16761858,
 11042306,
 15023106,
 16767746,
 11042818,
 15026690,
 16768258,
 11043330,
 15028226,
 16768770,
 11043842,
 15029762,
 16769282,
 11044354,
 15032322,
 9664514,
 11044866,
 15032834,
 16774658,
 11045378,
 15033346,
 16775426,
 11045890,
 15035138,
 16776450,
 11047938,
 15035906,
 16777474,
 11051778,
 15036418,
 16777986,
 11052290,
 15040770,
 16779010,
 11053314,
 15041282,
 16779522,
 11053826,
 15041794,
 16780034,
 11055362,
 15042306,
 16783106,
 11057410,
 15042818,
